# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [1]:
!pip install -U langchain langchain-core langchain-classic langchain-pinecone langchain-huggingface datasets pinecone-client sentence-transformers torch

  Using cached datasets-4.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached pinecone_client-6.0.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached huggingface_hub-0.36.2-py3-none-any.whl.metadata (15 kB)
  Using cached pyarrow-23.0.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.1 kB)
  Using cached pinecone_plugin_assistant-1.8.0-py3-none-any.whl.metadata (30 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-5.1.0-py3-none-any.whl.metadata (31 kB)
  Using cached transformers-5.0.0-py3-none-any.whl.metadata (37 kB)
  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
Using cached datasets-4.6.0-py3-none-any.whl (520 kB)
Using cached pinecone_client-6.0.0-py3-none-any.whl (6.7 kB)
Using cached huggingface_hub-0.36.2-py3-none-any.whl (566 kB)
Using cached pinecone-7.3.0-p

In [2]:
!pip uninstall -y pyarrow datasets numpy pandas transformers sentence-transformers huggingface_hub -q
!pip install numpy==1.26.4 "pyarrow==14.0.2" "datasets==2.18.0" transformers sentence-transformers huggingface_hub -U -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 3.0.1 which is incompatible.
opencv-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
dask-cudf-cu12 25.10.0 requires pandas<2.4.0dev0,>=2.0, but you have pandas 3.0.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.5 which is incompatible.
gradio 5.50.0 requires pandas<3.0,>=1.0, but you have pandas 3.0.1 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 

# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [3]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
from datasets import load_dataset
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
).shuffle(seed=960)

/usr/local/lib/python3.12/dist-packages/datasets/load.py:1461: FutureWarning: The repository for vblagoje/wikipedia_snippets_streamed contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/vblagoje/wikipedia_snippets_streamed
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [4]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'wiki_id': 'Q7649565',
 'start_paragraph': 20,
 'start_character': 272,
 'end_paragraph': 24,
 'end_character': 380,
 'article_title': 'Sustainable Agriculture Research and Education',
 'section_title': "2000s & Evaluation of the program's effectiveness",
 'passage_text': "preserving the surrounding prairies. It ran until March 31, 2001.\nIn 2008, SARE celebrated its 20th anniversary. To that date, the program had funded 3,700 projects and was operating with an annual budget of approximately $19 million. Evaluation of the program's effectiveness As of 2008, 64% of farmers who had received SARE grants stated that they had been able to earn increased profits as a result of the funding they received and utilization of sustainable agriculture methods. Additionally, 79% of grantees said that they had experienced a significant improvement in soil quality though the environmentally friendly, sustainable methods that they were"}

In [5]:
# The 'wiki_snippets' dataset does not have 'section_title', so we will proceed without this specific filter
history = wiki_data.filter(lambda x: x["section_title"] == "History")

Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [6]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 10000

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(history, total=total_doc_count):
    # extract the fields we need - article, section, and passage
    article_title = d['article_title']
    passage_text = d['passage_text']
    section_title = "" # 'wiki_snippets' does not have a section_title field

    # Only collect up to total_doc_count documents
    if counter < total_doc_count:
        docs.append({
            'article_title': article_title,
            'section_title': section_title,
            'passage_text': passage_text
        })
        counter += 1
    else:
        break # Stop after reaching the desired number of documents

  0%|          | 0/10000 [00:00<?, ?it/s]

In [7]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head()

,article_title,section_title,passage_text
0,Taupo District,,was not until the 1950s that the region starte...
1,The Bishop Wand Church of England School,,The Bishop Wand Church of England School Histo...
2,Surface Hill Uniting Church,,in perpetual reminder that work and worship go...
3,The Electras (band),,"as its B-side. However, copies of the single, ..."
4,Swanton House,,it. Lane provided funds for restoration by the...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [8]:
!pip uninstall -y pinecone-client pinecone -q
!pip install "pinecone[grpc]" -U -q  # Latest v5 with gRPC

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.2.13 requires pinecone[asyncio]<8.0.0,>=6.0.0, but you have pinecone 8.1.0 which is incompatible.


In [9]:
from google.colab import userdata

def get_pinecone_api_key():
    """Retrieves the Pinecone API key from Colab secrets."""
    # Ensure the secret is named 'PINECONE_API_KEY' in Colab secrets
    return userdata.get('PINECONE_API_KEY')

# Example usage:
pinecone_api_key = get_pinecone_api_key()


In [10]:
from google.colab import userdata
import os
from pinecone import Pinecone
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

# connect to pinecone environment
pc = Pinecone(api_key=pinecone_api_key, environment=spec.region)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [11]:
index_name = "abstractive-question-answering"

# Check if index exists & create with ServerlessSpec
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")  # ✅ REQUIRED for serverless
    )

# Connect to index
index = pc.Index(index_name)

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [12]:
# Already installed above
# import torch
# from sentence_transformers import SentenceTransformer

# set device to GPU if available
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
# retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base').to(device)
# retriever

In [13]:
import torch
from sentence_transformers import SentenceTransformer

# Set device (GPU if available)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load lab retriever (768-dim, cosine-optimized)
retriever = SentenceTransformer(
    'flax-sentence-embeddings/all_datasets_v3_mpnet-base',
    device=device
)

# Test
test_texts = ["What is machine learning?", "Machine learning definition"]
embeddings = retriever.encode(test_texts)
print(f"✅ Loaded retriever (shape: {embeddings.shape})")  # (2, 768)
retriever

Using device: cuda


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: flax-sentence-embeddings/all_datasets_v3_mpnet-base
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Loaded retriever (shape: (2, 768))


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [14]:
#!pip install sympy==1.13.3

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [15]:
from tqdm.auto import tqdm
import uuid

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch['passage_text'].tolist()).tolist()

    # Create Pinecone vectors with ID + metadata for this batch
    pinecone_vectors_batch = []
    for j, embedding in enumerate(emb):
        pinecone_vectors_batch.append({
            "id": str(uuid.uuid4()),  # Unique ID
            "values": embedding,      # Embedding vector
            "metadata": {
                "article_title": batch['article_title'].iloc[j],
                "section_title": batch['section_title'].iloc[j],
                "passage_text": batch['passage_text'].iloc[j]
            }
        })

    # Upsert the current batch to Pinecone
    index.upsert(vectors=pinecone_vectors_batch)

# check that we have all vectors in index
print(index.describe_index_stats())

  0%|          | 0/157 [00:00<?, ?it/s]

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '189',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 25 Feb 2026 21:49:26 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '40',
                                    'x-pinecone-request-latency-ms': '40',
                                    'x-pinecone-response-duration-ms': '41'}},
 'dimension': 768,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 40000}},
 'storageFullness': 0.0,
 'total_vector_count': 40000,
 'vector_type': 'dense'}


# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [16]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [17]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode([query]).tolist()[0]
    # search pinecone index for context passage with the answer
    xc = index.query(
        vector=xq,
        top_k=top_k,
        include_metadata=True
    )
    return xc

In [18]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['passage_text']}" for m in context]
    # concatinate all context passages
    context = ' '.join(context)
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [19]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

QueryResponse(matches=[{'id': '767ad4b9-a8cb-49d0-babc-b0294413689c',
 'metadata': {'article_title': 'Electric power system',
              'passage_text': 'Electric power system History In 1881, two '
                              "electricians built the world's first power "
                              'system at Godalming in England. It was powered '
                              'by two waterwheels and produced an alternating '
                              'current that in turn supplied seven Siemens arc '
                              'lamps at 250 volts and 34 incandescent lamps at '
                              '40 volts. However, supply to the lamps was '
                              'intermittent and in 1882 Thomas Edison and his '
                              'company, The Edison Electric Light Company, '
                              'developed the first steam-powered electric '
                              'power station on Pearl Street in New York City. '
          

In [20]:
from pprint import pprint

In [21]:
# format the query in the form generator expects the input
query = format_query(query, result['matches'])
pprint(query)

('question: when was the first electric power system built? context: <P> '
 "Electric power system History In 1881, two electricians built the world's "
 'first power system at Godalming in England. It was powered by two '
 'waterwheels and produced an alternating current that in turn supplied seven '
 'Siemens arc lamps at 250 volts and 34 incandescent lamps at 40 volts. '
 'However, supply to the lamps was intermittent and in 1882 Thomas Edison and '
 'his company, The Edison Electric Light Company, developed the first '
 'steam-powered electric power station on Pearl Street in New York City. The '
 'Pearl Street Station initially powered around 3,000 lamps for 59 customers. '
 'The power station generated direct current and')


The output looks great. Now let's write a function to generate answers.

In [22]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [23]:
generate_answer(query)

('The first electric power system was built in 1881 at Godalming in England. '
 'It was powered by two waterwheels and produced alternating current that in '
 'turn supplied seven Siemens arc lamps at')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [24]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The first wireless message was sent by a telegraph. The first telegraph was '
 'built in the late 1800s, and it was used to send messages from London to New '
 'York. The telegraph')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [25]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

audio distance records, and were heard as far west as Hawaii. They were also received in Paris, France, which marked the first transmission of speech across the Atlantic.
With the entrance of the United States into World War I in April 1917 the federal government took over full control of the radio industry, and it became illegal for civilians to possess an operational radio receiver. However NAA continued to operate during the conflict. In addition to time signals and weather reports, it also broadcast news summaries received by troops on land and aboard ships in the Atlantic. Effective April 15, 1919
---
audio distance records, and were heard as far west as Hawaii. They were also received in Paris, France, which marked the first transmission of speech across the Atlantic.
With the entrance of the United States into World War I in April 1917 the federal government took over full control of the radio industry, and it became illegal for civilians to possess an operational radio receiver

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [26]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('COVID-19 is a strain of the coronaviruses. The coronaviruses are a group of '
 'viruses that infect humans. The coronaviruses are a group of viruses that')


In [27]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

man killed and twenty-nine died of disease.
---
man killed and twenty-nine died of disease.
---
man killed and twenty-nine died of disease.
---
man killed and twenty-nine died of disease.
---
diseases when they occupy at certain times of the year natural habitat of a certain pathogen (plague, tularemia, leptospirosis, arboviruses, tick-borne relapsing fever. The WHO Expert Committee on Zoonoses listed over 100 such diseases. About natural focality of the diseases is known elsewhere. History Historically, Sanitary epidemiological reconnaissance implied collection and transfer of all data available on sanitary and epidemiological situation of the area of possible deployment and action of armed forces, the same data for the neighbouring and enemy armed forces. The aim for the reconnaissance was to clear up the reasons of the specific disease origin—sources of the
---


Let’s finish with a final few questions.

In [28]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The War of Currents was a rivalry between Thomas Edison and George '
 "Westinghouse's companies over which form of transmission (direct or "
 'alternating current) was superior, a series of events known as the')


In [29]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to walk on the moon was Giovanni Cassini. He walked on the '
 'moon in 1969.')


In [30]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The Space Shuttle was the most expensive project in the history of NASA. It '
 'cost about $10 billion to build.')


As we can see, the model can generate some decent answers.

#### Add a few more questions

In [31]:
query = "Who invented television?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

("I'm not sure if this is the right subreddit to ask this question, but I'll "
 "try to answer it anyway. I'm not a historian, but I do have a degree in "
 'computer science')


In [32]:
query = "Who designed the Sydney Opera house?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

('The Sydney Opera House was designed by Samuel Sloan, of Philadelphia, and '
 'his assistant, Aldophus Gustavus Bauer, were chosen as architects. Sloan '
 'arrived in Raleigh with his designs for the elaborate')


In [33]:
query = "How is Joghurt produced?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

("I'm not sure if this is what you're looking for, but I can tell you that "
 'yogurt is made by fermenting milk. The milk is then heated to a temperature '
 'where the lactose')
